<H1>SCN2A: Generate Transcript HGVS </H1>
<p>In this notebook, we generate nucleotide HGVS expressions for the variants listed in 
  <a href="https://pubmed.ncbi.nlm.nih.gov/33731876/" target="__blank">Crawford et al, Computational analysis of 10,860 phenotypic annotations in individuals with SCN2A-related disorders</a>. In the Supplementary Table of this publication, variants were given in Protein notation. However, for our software, we require transcript level HGVS notations. We used the <a href="http://useast.ensembl.org/Homo_sapiens/Tools/VR" target="__blank">Variant Recoder</a> tool of Ensembl to extract the corresponding data and here create a mapping file that will be used in the other SCN2A notebook to generate phenopackets.</p>
  <p>Note that <a href="https://useast.ensembl.org/Homo_sapiens/Transcript/Summary?db=core;g=ENSG00000136531;r=2:165194993-165392310;t=ENST00000375437" target="__blank">ENST00000375437.7</a> is the MANE select transcript that we use here. It corresponds to <a href="https://www.ncbi.nlm.nih.gov/nuccore/NM_001040142.2" target="__blank">NM_001040142.2</a>.</p>
 <p>The purpose of this notebook is to prepare HGVS-compliant variant strings that we will use in the Crawford_SCN2A notebook to generate phenopackets.</p>

In [1]:
import pandas as pd
from collections import defaultdict

In [2]:
vr_df = pd.read_csv("input/vr_output", comment='#', delimiter='\t')

In [3]:
vr_df.head()

,Uploaded_variant,Allele,HGVSg,HGVSc,HGVSp,VARID,Variant_synonyms,MANE_Select,VCF,HGVSg.1,HGVSc.1,HGVSp.1,VARID.1,MANE_Select.1,VCF.1
0,ENSP00000364586:p.V423L,T,NC_000002.12:g.165313992G>T,"ENST00000283256.10:c.1267G>T, ENST00000375437....","ENSP00000283256.6:p.Val423Leu, ENSP00000364586...",CM176086,-,NC_000002.12:g.165313992G>T;ENST00000375437.7:...,2-165313992-G-T,NaN,NaN,NaN,NaN,NaN,NaN
1,ENSP00000364586:p.L501*,TA,NC_000002.12:g.165315588_165315589delinsTA,"ENST00000283256.10:c.1501_1502delinsTA, ENST00...","ENSP00000283256.6:p.Leu501Ter, ENSP00000364586...",-,-,NC_000002.12:g.165315588_165315589delinsTA;ENS...,2-165315588-CT-TA,NaN,NaN,NaN,NaN,NaN,NaN
2,ENSP00000364586:p.R583*,T,NC_000002.12:g.165323231C>T,"ENST00000283256.10:c.1747C>T, ENST00000375437....","ENSP00000283256.6:p.Arg583Ter, ENSP00000364586...","rs1553571901, CM154971, COSV51841058","dbSNP HGVS: NP_001035232.1:p.Arg583Ter,NM_0010...",NC_000002.12:g.165323231C>T;ENST00000375437.7:...,2-165323231-C-T,NaN,NaN,NaN,NaN,NaN,NaN
3,ENSP00000364586:p.D609*,TAA,NC_000002.12:g.165323309_165323311delinsTAA,"ENST00000283256.10:c.1825_1827delinsTAA, ENST0...","ENSP00000283256.6:p.Asp609Ter, ENSP00000364586...",-,-,NC_000002.12:g.165323309_165323311delinsTAA;EN...,2-165323309-GAC-TAA,NaN,NaN,NaN,NaN,NaN,NaN
4,ENSP00000364586:p.D609*,TAG,NC_000002.12:g.165323309_165323311delinsTAG,"ENST00000283256.10:c.1825_1827delinsTAG, ENST0...","ENSP00000283256.6:p.Asp609Ter, ENSP00000364586...",-,-,NC_000002.12:g.165323309_165323311delinsTAG;EN...,2-165323309-GAC-TAG,NaN,NaN,NaN,NaN,NaN,NaN


<h3>Variant Record Mappings</h3>
<p>The following code creates a mapping such as D195G: NM_001040142.2:c.584A>G</p>

In [4]:
protein_to_NM_MANE_d = defaultdict()

In [24]:
for _, row in vr_df.iterrows():
    # Remove the prefix, i.e., ENSP00000364586:p.
    # this will regenerate the String used in the input file
    uploaded_var = row['Uploaded_variant'][18:]  
    mane_fields = row['MANE_Select'].split(";")
    nm_mane_filter_obj = filter(lambda t: t.startswith("NM_001040142"), mane_fields)
    nm_mane = list(nm_mane_filter_obj)[0]
    nm_mane = nm_mane.replace("NM_001040142.2:","")
    if nm_mane is None:
        raise ValueError(f"Could not find NM_ for {mane_fields}")
    protein_to_NM_MANE_d[uploaded_var] = nm_mane
print(f"We extracted {len(protein_to_NM_MANE_d)} mappings")

We extracted 243 mappings


<h3>Input the Supplemental file</h3>
<p>Only some of the input variants are missense. We need to manually map the remaining variants.</p>

In [25]:
df = pd.read_csv("input/41436_2021_1120_MOESM2_ESM.csv", delimiter='\t')

In [26]:
df.head()

,famID,broad_phx,variant,variant_type_1,variant_type_2,location,domain,segment,pmid/pmcid,HPO,hpo_term
0,fam1,benign,A202V,missense,missense,Helical Repeat I,DI,S3,28379373,HP:0007359,Focal-onset seizure
1,fam1,benign,A202V,missense,missense,Helical Repeat I,DI,S3,28379373,HP:0002069,Generalized tonic-clonic seizures
2,fam1,benign,A202V,missense,missense,Helical Repeat I,DI,S3,28379373,NP:0003808,No Abnormal muscle tone
3,fam1,benign,A202V,missense,missense,Helical Repeat I,DI,S3,28379373,NP:0012443,No Abnormality of brain morphology
4,fam1,benign,A202V,missense,missense,Helical Repeat I,DI,S3,28379373,NP:0000717,No Autism


In [27]:
fam_to_var_d = defaultdict()
for _, row in df.iterrows():
    famID = row["famID"]
    variant = row["variant"]
    fam_to_var_d[famID] = variant
print(f"We extracted {len(fam_to_var_d)} families")

We extracted 414 families


<h2>Extracting variants that could not be mapped by Variant Report</h2>
<p>The variants in <tt>manual_lookups</tt> were looked up manually in the original publications.</p>
<p>The variants in the <tt>malformed</tt> were discarded since the nucleotide/amino acid changes because
information about the nuceltodie changes could not be found or because the variants
do not match the prefered transcript and
no transcript was provided in the original publication.</p>

In [28]:
# These are based on NM_001040142.2
manual_lookups = {

   "A202V": ["c.605C>T","NM_001040142.2(SCN2A):c.605C>T (p.Ala202Val)"],
    "N503K*19": ["c.1508dup", "NM_001040142.2:c.1508dup NP_001035232.1:p.(Asn503LysfsTer19)"],
    "L611V*35": ["c.1831_1832del", "NM_001040142.2:c.1831_1832del  NP_001035232.1:p.(Leu611ValfsTer35)"],
    "E440R*20": ["c.1318_1349del", "NM_001040142.2:c.1318_1349del  NP_001035232.1:p.(E440Rfs*20)"],
    "F1861L*40": ["c.5583del", "NM_001040142.2:c.5583del  NP_001035232.1:p.(F1861Lfs*40)"],
    "M1797I*5": ["c.5387_5390dup", "NM_001040142.2:c.5387_5390dup NP_001035232.1:p.(M1797Ifs*5)"],
    "Q1803G": ["c.5408A>G", "NM_001040142.2:c.5408A>G  NP_001035232.1:p.(E1803G)"],
    "Q169G": ["c.506A>G", "NM_001040142.2:c.506A>G  NP_001035232.1:p.(E169G)"],
    "I1537_M1538delinsSI": ["c.4610_4614delinsGCATC", "NM_001040142.2:c.4610_4614delinsGCATC NP_001035232.1:p.(I1537_M1538delinsSI)"],
     "T784C*45" : ["c.2350_2365delinsTGTACTATCCAACAGATACT", "NM_001040142.2:c.2350_2365delinsTGTACTATCCAACAGATACT  NP_001035232.1:p.(T784Cfs*45)"],
    "I874M*5": ["c.2622_2631del", "NM_001040142.2(SCN2A):c.2622_2631del NP_001035232.1:p.(Ile874MetfsTer5) NP_001035232.1:p.(I874Mfs*5)"],
    "K1387S*4": ["c.4160_4161del", "NM_001040142.2(SCN2A):c.4160_4161del (p.Lys1387fs)"],
    "I1615R*47": ["c.4844_4845del", "NM_001040142.2(SCN2A):c.4844_4845del (p.Ile1615fs) "]

}

malformed = { 
    "G999L",
    "V1261M",
    "V1263M",
    "V1528C*7",
    "Q1904R*22",
    "Q430G",
    "L392L", # splice but no nt available
    "F1599C*14", # original article ambiguous
    "N361T*21", # cannot find
    "T435I*5", # cannot find
    "C1170V*15", # cannot find
    "T1711L*8",     # cannot find
    "K1260E;K1260Q",
    "[R1882G;G1522A]",
    "Del(exons_18-27)",
    "missense",
    "2q24.3 duplication"           
}

In [43]:
mappings = []
for famID, var in fam_to_var_d.items():
    #print(famID, var)
    if var.startswith("c."):
        mane = var
    elif var == '4551+1C>A':
        mane = "c.4551+1C>A"
    elif var == "c.2379+1G>A":
        mane = "c.2388+1G>A"  # corrected from PMID:26647175 on basis of chromosomal position
    elif var in protein_to_NM_MANE_d:
        mane = protein_to_NM_MANE_d.get(var)
    elif var in manual_lookups:
        mane = manual_lookups.get(var)[0]
    elif var in  malformed:
        continue
    else:
        print(f"Could not find {var}")
        continue
    m = {'famID' : famID, 'var': var, 'MANE': mane}
    mappings.append(m)
print(f"Got {len(mappings)} mappings")
mapping_df = pd.DataFrame(mappings)

Got 396 mappings


In [44]:
mapping_df.head()

,famID,var,MANE
0,fam1,A202V,c.605C>T
1,fam10,V423L,c.1267G>T
2,fam100,L501*,c.1501_1502delinsTA
3,fam101,N503K*19,c.1508dup
4,fam102,R583*,c.1747C>T


In [45]:
mapping_df.to_csv("variant_mapping.tsv", index=False)